# Imports

In [2]:
import requests
import pandas as pd
import time

# API Scraping Function

In [3]:
def get_title(subreddit, pulls):
    df = pd.DataFrame()
    url = 'https://api.pushshift.io/reddit/search/submission'
    if subreddit == 'todayilearned':
        important = ['title', 'created_utc', 'removed_by_category', 'score', 'subreddit']
        #r/TIL keeps up moderated content while r/STIL does not. 'removed_by_category' will allow us to delete those posts later
    else:
        important = ['title', 'created_utc', 'score', 'subreddit']
    params = {
            'subreddit' : subreddit, 
            'size' : 100,
            'before' : 2000000000
            #'before' is set to a date in the future, so it will first grab the newest posts
            }
    for i in range(pulls):
        res = requests.get(url, params)
        data = res.json()
        posts = data['data']
        posts_frame = pd.DataFrame(posts)
        #puts the posts in a temporary dataframe
        df = df.append(posts_frame[important])
        #adds the temporary mainframe to the main dataframe
        params['before'] = posts_frame['created_utc'].min()
        #looks at the oldest post and changes 'before' to that value. Next pull will look at the newest posts that are older than that last post
        time.sleep(1)
        #just to make sure I don't send too many pull requests too quickly
    df.to_csv(f'../data/{subreddit}.csv', index=False)
    #no return value necessary. Just want to save that dataframe.

# Function Use

In [3]:
get_title('shittytodayilearned', 79)

This will grab the entire post history of r/STIL

In [4]:
get_title('todayilearned', 1_250)

This amount is what's necessary to grab enough r/TIL posts for method 2